In [2]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [3]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from nltk import FreqDist
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import  ward, dendrogram
from gensim import corpora, models
import gensim
from gensim.models.doc2vec import *
from sklearn.pipeline import Pipeline

Populating the interactive namespace from numpy and matplotlib


In [4]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:31, 2198.70it/s]


#  Домашнее задание по NLP # 2 [100 баллов] 
## Составление словарей для классификации по тональности
При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.  

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).


Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

In [5]:
responses[99]

{'city': 'г. Саратов',
 'rating_not_checked': False,
 'title': 'Карта ко вкладу',
 'num_comments': 0,
 'bank_license': 'лицензия № 880',
 'author': 'ronnichka',
 'bank_name': 'Югра',
 'datetime': '2015-06-03 20:56:57',
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'rating_grade': 3}

## Часть 1. Обучение модели word2vec [50 баллов]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.  
 


Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [6]:
texts = []
for i in range(len(responses)):
    texts.append(responses[i]['text'])

In [7]:
regex = re.compile("[А-Яа-я]+")
def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return [l for l in m.lemmatize(text) if l.isalpha()]
    except:
        return []

def preprocessing(text):
    words = words_only(text)
    no_stopwords = remove_stopwords(words)
    lemmas = lemmatize(no_stopwords)
    return(lemmas)

In [8]:
df = pd.DataFrame(texts)

In [9]:
df.columns = ['text']

In [10]:
%time df['lemmas'] = df.text.apply(lambda x: preprocessing(x))

CPU times: user 2min 25s, sys: 4.23 s, total: 2min 29s
Wall time: 13min 41s


In [11]:
%%time

model = Word2Vec(df.lemmas, size=100, window=3, min_count=20, workers=4)

CPU times: user 3min 37s, sys: 998 ms, total: 3min 38s
Wall time: 1min 7s


In [12]:
model.save("word2v.model")
model = Word2Vec.load("word2v.model")

In [13]:
model.most_similar("сайт")

[('портал', 0.5969380736351013),
 ('форум', 0.5776290893554688),
 ('страничка', 0.575488805770874),
 ('раздел', 0.5745071768760681),
 ('стенд', 0.5603625178337097),
 ('страница', 0.5070267915725708),
 ('лайн', 0.4931655824184418),
 ('твиттер', 0.48876383900642395),
 ('фейсбук', 0.48694050312042236),
 ('онлайн', 0.4850178360939026)]

In [14]:
# Модель word2vec хорошо оперделила близкие слова к слову "сайт"

In [15]:
model.most_similar(positive=["королева", "мужчина"], negative=["король"])

[('южнобутовский', 0.5597969889640808),
 ('киров', 0.5553302764892578),
 ('челюскинец', 0.5469536185264587),
 ('киевский', 0.5382822751998901),
 ('лесков', 0.5274077653884888),
 ('омск', 0.5269875526428223),
 ('донской', 0.5266583561897278),
 ('аврора', 0.5264283418655396),
 ('калинин', 0.5254647731781006),
 ('магнитогорск', 0.5209793448448181)]

In [16]:
# аналогии по корпусу слов определяет. Женщина - 3-я в списке

In [17]:
model.doesnt_match("москва жуковский омск женщина".split())

'женщина'

In [18]:
# аналогичным образом хорошо в модели работает тест на определение лишнего слова

In [19]:
top_words = []

fd = FreqDist()
for text in tqdm(df.lemmas):
    fd.update(text)
for i in fd.most_common(500):
    top_words.append(i[0])
print(top_words[:15])

100%|██████████| 153499/153499 [00:17<00:00, 8725.73it/s] 

['банк', 'карта', 'я', 'деньги', 'день', 'мой', 'в', 'кредит', 'который', 'отделение', 'клиент', 'сотрудник', 'счет', 'сказать', 'сумма']


In [20]:
top_words = [w for w in top_words if len(w) > 4]
len(top_words)

398

In [21]:
top_words_vec = model[top_words]

In [27]:
tsne = TSNE(n_components=2, random_state=0)

%time top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 2.3 s, sys: 7.76 ms, total: 2.3 s
Wall time: 1.85 s


In [28]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [32]:
pca = PCA(n_components=2)
%time pca_words = pca.fit_transform(top_words_vec)

CPU times: user 57.1 ms, sys: 0 ns, total: 57.1 ms
Wall time: 19.8 ms


In [33]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec PCA for most common words")

source = ColumnDataSource(data=dict(x1=pca_words[:,0],
                                    x2=pca_words[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [30]:
horminploho = model['хорошо'] - model['плохо']

In [31]:
bistrominmedlenno = model['быстро'] - model['медленно']

In [34]:
sberx = horminploho*model['сбербанк'] 

In [35]:
sbery = bistrominmedlenno*model['сбербанк']

In [37]:
sum(sberx)

7.231163

In [38]:
sum(sbery)

8.468031

In [49]:
banki = \
[
  'сбербанк',
  'альфа',
  'втб',
  'хоум',
  'тинькофф',
  'траст',
  'ренессанс',
  'связной', 
  'отп'
]

In [50]:
bank_coord = []
for i in banki:
    x = sum(horminploho*model[i])
    y = sum(bistrominmedlenno*model[i])
    bank_coord.append([i, x, y])

In [51]:
bank_coord

[['сбербанк', 7.231163, 8.468031],
 ['альфа', -2.6784751, 16.48028],
 ['втб', -8.30411, 7.8610134],
 ['хоум', -7.16312, 5.8462744],
 ['тинькофф', -7.272961, 19.209276],
 ['траст', 0.022871614, -5.1952953],
 ['ренессанс', 1.6567118, 16.925434],
 ['связной', 6.9035625, 18.553318],
 ['отп', -1.3487852, -0.47511053]]

In [61]:
names = []
x = [] 
y = [] 
for i in bank_coord:
    names.append(i[0])
    x.append(i[1])
    y.append(i[2])

In [64]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec coords of banks on axis good-bad, fast-slow")

source = ColumnDataSource(data=dict(x1=x,
                                    x2=y,
                                    names=names))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

## Часть 2. Распространение метки [50 баллов]

Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8  негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
1. ```graphlab.label_propagation``` (```graphlab``` доступен бесплатно по образовательной лицензии)
2. ```sklearn.semi_supervised.LabelPropagation``` 
3. ```sklearn.semi_supervised.LabelSpreading```

In [73]:
# пример построения графа 

import igraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.

In [69]:
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

NameError: name 'g' is not defined